In [2]:
import numpy as np
import scipy.io as sio
import pywt

In [2]:
# Filter bank for the 3-band 2-regular wavelet
wv32 = (
    ( 0.33838609728386,  0.53083618701374,  0.72328627674361,  0.23896417190576,  0.04651408217589, -0.14593600755399),
    (-0.11737701613483,  0.54433105395181, -0.01870574735313, -0.69911956479289, -0.13608276348796,  0.42695403781698),
    ( 0.40363686892892, -0.62853936105471,  0.46060475252131, -0.40363686892892, -0.07856742013185,  0.24650202866523)
)

# Filter bank for the 4-band 2-regular wavelet
wv42 = (
    (-0.067371764,  0.094195111,  0.40580489 ,  0.567371764,  0.567371764,  0.40580489 ,  0.094195111, -0.067371764),
    (-0.094195111,  0.067371764,  0.567371764,  0.40580489,  -0.40580489 , -0.567371764, -0.067371764,  0.094195111),
    (-0.094195111, -0.067371764,  0.567371764, -0.40580489,  -0.40580489 ,  0.567371764, -0.067371764, -0.094195111),
    (-0.067371764, -0.094195111,  0.40580489 , -0.567371764,  0.567371764, -0.40580489 ,  0.094195111,  0.067371764)
)

# Filter bank for the 4-band 4-regular wavelet
wv44 = (
    ( 0.0857130200,  0.1931394393,  0.3491805097,  0.5616494215,  0.4955029828,  0.4145647737,  0.2190308939, -0.1145361261,
     -0.0952930728, -0.1306948909, -0.0827496793,  0.0719795354,  0.0140770701,  0.0229906779,  0.0145382757, -0.0190928308),
    (-0.1045086525,  0.1183282069, -0.1011065044, -0.0115563891,  0.6005913823, -0.2550401616, -0.4264277361, -0.0827398180,
      0.0722022649,  0.2684936992,  0.1691549718, -0.4437039320,  0.0849964877,  0.1388163056,  0.0877812188, -0.1152813433),
    ( 0.2560950163, -0.2048089157, -0.2503433230, -0.2484277272,  0.4477496752,  0.0010274000, -0.0621881917,  0.5562313118,
     -0.2245618041, -0.3300536827, -0.2088643503,  0.2202951830,  0.0207171125,  0.0338351983,  0.0213958651, -0.0280987676),
    ( 0.1839986022, -0.6622893130,  0.6880085746, -0.1379502447,  0.0446493766, -0.0823301969, -0.0923899104, -0.0233349758,
      0.0290655661,  0.0702950474,  0.0443561794, -0.0918374833,  0.0128845052,  0.0210429802,  0.0133066389, -0.0174753464)
)

In [3]:
def dwt_matrix(filter_banks, rows_per_filter_bank):
    # Assertions that may or may not be needed
    assert len(filter_banks) > 0
    filter_bank_length = len(filter_banks[0])
    assert all(len(filter_bank) == filter_bank_length for filter_bank in filter_banks)
    assert filter_bank_length % len(filter_banks) == 0 # Not sure if this must be true
    # Mk x Mk matrix
    matrix_size = len(filter_banks) * rows_per_filter_bank
    assert matrix_size >= filter_bank_length

    shift = len(filter_banks) # I don't know if this is true
    res = np.zeros((matrix_size, matrix_size))
    row = 0
    for filter_bank in filter_banks:
        current_offset = 0
        for _ in range(rows_per_filter_bank):
            for (i, value) in enumerate(filter_bank):
                res[row][(current_offset + i) % matrix_size] = value
            current_offset += shift
            row += 1
    return res

def dwt_2d(wavelet, signal):
    bands = len(wavelet)
    assert len(signal) % len(wavelet) == 0
    assert np.shape(signal)[0] == np.shape(signal)[1]
    k = len(signal) // len(wavelet)
    matrix = dwt_matrix(wavelet, k)
    values = np.matmul(matrix, np.matmul(signal, np.transpose(matrix)))
    return np.array(tuple(np.split(row, bands) for row in np.split(values, bands, 1)))

In [4]:
def decomposeImageUsingWavelet(unprocessed, foldername, wavelet, waveletname):
    preprocessed = np.array([np.array([dwt_2d(wavelet, band) for band in image]) for image in unprocessed])
    for j in range(preprocessed.shape[2]):
        for k in range(preprocessed.shape[3]):
            mdic = {"imgs" : preprocessed[:, :, j, k, :, :]}
            sio.savemat(f"../data/{foldername}/wavelet-decomposed-py/{waveletname}/band_{j}_{k}.mat", mdic)

def decomposeImage(foldername):
    unprocessed = sio.loadmat(f"../data/{foldername}/unprocessed_lakes.mat")['imgs']

    for i in range(2, 6):
        dbi = pywt.Wavelet(f'db{i}').filter_bank[2:4]
        decomposeImageUsingWavelet(unprocessed, foldername, dbi, f'db{i}')

    decomposeImageUsingWavelet(unprocessed, foldername, wv32, "wv32")
    decomposeImageUsingWavelet(unprocessed, foldername, wv42, "wv42")
    decomposeImageUsingWavelet(unprocessed, foldername, wv44, "wv44")

In [5]:
%%time

decomposeImage("thermokarst_lakes_preprocessed")
decomposeImage("glacial_lakes_preprocessed")

CPU times: user 3min 22s, sys: 5min 39s, total: 9min 1s
Wall time: 2min 33s
